In [5]:
import pandas as pd
import numpy as np


In [6]:
products = pd.read_csv('products_train.csv')
unique_locales = products['locale'].unique()

In [7]:
## LOCALES TO TRANSLATE
unique_locales = np.delete(unique_locales, np.where(unique_locales == 'UK'))
unique_locales

array(['DE', 'JP', 'ES', 'FR', 'IT'], dtype=object)

# German Machine Translation

reference: [English To German Model](https://huggingface.co/google/bert2bert_L-24_wmt_de_en)

## Sample example

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/bert2bert_L-24_wmt_de_en", pad_token="<pad>", eos_token="</s>", bos_token="<s>", unk_token="<unk>")
model = AutoModelForSeq2SeqLM.from_pretrained("google/bert2bert_L-24_wmt_de_en")

In [9]:
## SILLY EXAMPLE, CAN PLAY WITH THIS IF YOU WANT :)
sentence = "Willst du einen как trinken gehen mit mir?"

input_ids = tokenizer(sentence, return_tensors="pt", add_special_tokens=False).input_ids
output_ids = model.generate(input_ids)[0]
print(tokenizer.decode(output_ids, skip_special_tokens=True))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (128) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Want to drink a как go with me?


## Real fun part

In [10]:
def process(text):
    import re    # for regular expressions
    from nltk.stem import WordNetLemmatizer
    from nltk.tokenize import word_tokenize
    if isinstance(text, str):
        text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
        text = re.sub(r'[^\w\s]', "", text)
        text = re.sub(r'[^\w\d\s]', '', text)
        ## THE TEXT CONTAINED THE SAME GARBAGE AS IN ENGLISH TITLES
        ## SHIT DATASET
        # I HATE LIFE
        text = re.findall(r'\b(?:[a-zA-Z]+|\d+)\b', text)
        text = " ".join(text)
        # removing non-unicode characters
        # for example, it will remove character 'Ç'
        non_ascii_pattern = re.compile(r'[^\x00-\x7F]+')
        text = non_ascii_pattern.sub('', text)

        # Tokenize it
        return text
    else:
        return ''

    # Return a string

In [11]:
## Get all the german rows
locale = 'DE'
de_products = products.loc[products['locale'] == locale]
# Get only titles
de_products_title = products['title']
# drop all nan rows
de_products_title.dropna()

0          RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...
1          Simply Keto Lower Carb* Schokodrops ohne Zucke...
2          Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...
3          AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...
4              PLAYMOBIL - 70522 - Cavaliere mit grauem Pony
                                 ...                        
1551052    Barbie - Playset Gelateria con Bambola con Mac...
1551053    Braun Silk-épil 1 Depilatore Donna, Epilatore ...
1551054    BoxLegend Sacchetti Sottovuoto Vestiti 6 Pezzi...
1551055    Trasportino Pratiko Metal - Accessorio da viag...
1551056    LiCB - Batterie LR1130, batterie alcaline AG10...
Name: title, Length: 1551049, dtype: object

In [12]:
# a bit of preprocessing
# will update it
de_products_title = de_products_title.apply(process)
# converting to list to translate the rows
de_products_title_list = de_products_title.tolist()

In [13]:
### DUMB DEBUGGER

## REAL FUN
## NOT FUN ACTUALLY
# decoded_sequences = []
# for i in range(161+102, len(de_products_title_list)):
#     input_ids = tokenizer(de_products_title_list[i], padding=True, truncation=True, return_tensors="pt", add_special_tokens=False).input_ids
#     output_ids = model.generate(input_ids)[0]
#     decoded_sequences.append(tokenizer.decode(output_ids, skip_special_tokens=True))


In [29]:
## REAL REAL FUN
## CHECK OUTPUT :)
input_ids = tokenizer(de_products_title_list, padding=True, return_tensors="pt", add_special_tokens=False).input_ids

In [ ]:
output_ids = model.generate(input_ids)

In [3]:
decoded_sequences = tokenizer.batch_decode(output_ids, skip_special_tokens = True)

NameError: name 'tokenizer' is not defined

In [1]:

translated = pd.DataFrame(decoded_sequences, columns=['title'])
translated.to_csv('german-to-english.csv')

NameError: name 'pd' is not defined

In [2]:
print(decoded_sequences)

NameError: name 'decoded_sequences' is not defined